# ДОМАШНЯЯ РАБОТА - "КОЛЛАБОРАТИВНАЯ ФИЛЬТРАЦИЯ"

ДАНИЛЬЧЕНКО ВАДИМ

ПАКЕТ SURPRISE

+ используйте данные MovieLens 1M
+ можно использовать любые модели из пакета
+ получите RMSE на тестовом сете 0.87 и ниже

In [2]:
from surprise import KNNWithMeans, KNNBasic, KNNWithZScore, KNNBaseline
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

import pandas as pd

In [3]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

In [4]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [5]:
movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

In [7]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [8]:
dataset.head()

,uid,iid,rating
0,7.0,Toy Story (1995),3.0
1,9.0,Toy Story (1995),4.0
2,13.0,Toy Story (1995),5.0
3,15.0,Toy Story (1995),2.0
4,19.0,Toy Story (1995),3.0


In [166]:
cnt_ratings_per_user=dataset.groupby('uid').rating.count().reset_index().rename(columns={'rating':'cnt_ratings'})
cnt_ratings_per_user[cnt_ratings_per_user.cnt_ratings<5].uid.count()

0

In [168]:
cnt_ratings_per_user.describe()

,uid,cnt_ratings
count,671.000000,671.000000
mean,336.000000,149.037258
std,193.845299,231.226948
min,1.000000,20.000000
25%,168.500000,37.000000
50%,336.000000,71.000000
75%,503.500000,161.000000
max,671.000000,2391.000000


удалим фильмы с количеством оценок менее 3х

In [176]:
cnt_ratings_per_movie=dataset.groupby('iid').rating.count().reset_index().rename(columns={'rating':'cnt_ratings'})
iid_to_excl=list(cnt_ratings_per_movie[cnt_ratings_per_movie.cnt_ratings<3].iid)
len(iid_to_excl)

4263

In [173]:
cnt_ratings_per_movie.describe()

,cnt_ratings
count,9064.000000
mean,11.033098
std,24.053106
min,1.000000
25%,1.000000
50%,3.000000
75%,9.000000
max,341.000000


In [177]:
ratings[~ratings.movieId.isin(iid_to_excl)].rating.min()

0.5

In [178]:
ratings[~ratings.movieId.isin(iid_to_excl)].rating.max()

5.0

In [180]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset[~dataset.iid.isin(iid_to_excl)], reader)

In [181]:
data

In [182]:
trainset, testset = train_test_split(data, test_size=.15, random_state =777 )

In [183]:
trainset.all_items()

range(0, 4798)

используем алогоритм KNNBaseline

In [184]:
algo = KNNBaseline(k=43, min_k=8, sim_options={'name': 'pearson_baseline', 'user_based': True})
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [185]:
test_pred = algo.test(testset)

In [186]:
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8646


0.8646366397811811

In [ ]:
# немного попыток до удаления фильмов с количеством оценок менее 3
"""
sim_options={'name': 'cosine', 'user_based': True, 'shrinkage': 50}: RMSE: 0.9343
sim_options={'name': 'pearson_baseline', 'user_based': True, 'shrinkage': 50}: RMSE: 0.9406 
sim_options={'name': 'pearson_baseline', 'user_based': True, 'shrinkage': 0}: RMSE: 0.9508
sim_options={'name': 'pearson_baseline', 'user_based': True, 'shrinkage': 100, 'verbose':False}: RMSE: 0.9409
k=50, min_k=5, sim_options={'name': 'pearson_baseline', 'user_based': True, 'shrinkage': 100}: RMSE: 0.9254
"""

In [189]:
algo.predict(uid=2, iid='Fight Club (1999)')

Prediction(uid=2, iid='Fight Club (1999)', r_ui=None, est=4.319689506061958, details={'actual_k': 43, 'was_impossible': False})

In [190]:
algo.get_neighbors(iid=3, k=3)

[97, 37, 350]